### Total theoretical peptides (FASTA)    

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!python --version

Python 3.10.12


In [8]:
# import libraries
import numpy as np
import pandas as pd
import re
!pip install Bio
from Bio import SeqIO
import csv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 16.8 MB/s eta 0:00:00


### Import df_ones_sorted_PTEN file

In [ ]:
### File obtained from Ahmad - 'https://github.com/i-am-robot3930/Tryptic-Cleavage'
fasta = pd.read_table('/content/drive/MyDrive/Colab Notebooks/peptide ml detection/data/df_ones_sorted_PTEN.csv', header=None)
fasta.head()

,0
0,"ProteinID,start,end,cleavage_sites,Cleavage,Pr..."
1,"P61981,70,77,TSADGNEK,0,1,0.500"
2,"Q8TBX8,366,382,EVYFMGLIDILTQYDAK,0,1,0.500"
3,"P50213,217,223,EVAESCK,0,1,0.500"
4,"Q96PZ0,189,201,ETSVAIEVIEDTK,0,1,0.500"


In [ ]:
### changes made to column names, from protein id to protein and peptides
### columns with missed cleavage value of '1' were filtered and saved as tsv file
df_ones_sorted_ones = pd.read_table('/content/df_ones_sorted_PTEN final.tsv')
df_ones_sorted_ones.head()

,Protein,Peptide,Cleavage,Prediction,Probability
0,P18031,EFEQIDK,1,1,0.5
1,O15371,EEMDFPQLMK,1,1,0.5
2,Q15084,DVIELTDDSFDK,1,1,0.5
3,P0DMV8,FGDPVVQSDMK,1,1,0.5
4,P48735,DIFQEIFDK,1,1,0.5


In [ ]:
### file renamed
fasta_proteins_new = pd.read_table('/content/fasta_proteins.tsv')
fasta_proteins_new.head()

,Protein,Sequence
0,Q66K14,MWLSPEEVLVANALWVTERANPFFVLQRRRGHGRGGGLTGLLVGTL...
1,Q9UMR3,MEFTASPKPQLSSRANAFSIAALMSSGGSKEKEATENTIKPLEQFV...
2,Q9P031,MAPVRRSAKWRPGGIEARGEGVSTVGYRNKNVRQKTWRPNHPQAFV...
3,Q6PEY2,MRECISIHVGQAGVQIGNACWELYCLEHGIQPDGQMPSDKTIGGGD...
4,Q9P016,MSRPRKRLAGTSGSDKGLSGKRTKTENSGEALAKVEDSNPQKTSAT...


In [ ]:
### Add sequences column to calculate their length and add them into 'sequence_length' column (i.e, sequence length for each protein should be calculated and those values are to be added to a 'sequence_length' column.
### This is done to calculate protein quantitation metrics later

import pandas as pd

# list_of_sequence = ["" for i in range (len(df_ones_sorted_ones. index))]
# list_of_sequence_length = [0 for i in range (len(df_ones_sorted_ones. index))]
list_of_sequence=[]
list_of_sequence_length=[]
# df_ones_sorted_ones = df_ones_sorted_ones.assign(sequence = list_of_sequence)
# df_ones_sorted_ones = df_ones_sorted_ones.assign(sequence_length = list_of_sequence_length)
for index, row in df_ones_sorted_ones.iterrows():
  new_row = fasta_proteins_new.loc[fasta_proteins_new["Protein"] == row["Protein"]]
  if(new_row.size == 0):
    list_of_sequence.append("")
    list_of_sequence_length.append(0)
    continue;
  list_of_sequence.append(new_row["Sequence"].values[0])
  list_of_sequence_length.append(len(new_row["Sequence"].values[0]))
  # row["sequence_length"] = len(new_row["Sequence"].values[0])
  # row["sequence"] = new_row["Sequence"].values[0]
  print(len(new_row["Sequence"].values[0]))
  # break
df_ones_sorted_ones = df_ones_sorted_ones.assign(sequence = list_of_sequence)
df_ones_sorted_ones = df_ones_sorted_ones.assign(sequence_length = list_of_sequence_length)
df_ones_sorted_ones.head()
df_ones_sorted_ones.to_csv('newfile.tsv', sep = '\t')

In [ ]:
fasta_proteins = pd.read_table('/content/newfile.tsv')
fasta_proteins.head()

,Unnamed: 0,Protein,Peptide,Cleavage,Prediction,Probability,sequence,sequence_length
0,0,P18031,EFEQIDK,1,1,0.5,MEMEKEFEQIDKSGSWAAIYQDIRHEASDFPCRVAKLPKNKNRNRY...,435
1,1,O15371,EEMDFPQLMK,1,1,0.5,MAKFMTPVIQDNPSGWGPCAVPEQFRDMPYQPFSKGDRLGKVADWT...,548
2,2,Q15084,DVIELTDDSFDK,1,1,0.5,MALLVLGLVSCTFFLAVNGLYSSSDDVIELTPSNFNREVIQSDSLW...,440
3,3,P0DMV8,FGDPVVQSDMK,1,1,0.5,MAKAAAIGIDLGTTYSCVGVFQHGKVEIIANDQGNRTTPSYVAFTD...,641
4,4,P48735,DIFQEIFDK,1,1,0.5,MAGYLRVVRSLCRASGSRPAWAPAALTAPTSQEQPRRHYADKRIKV...,452


#### To verify if the missed cleavages in the written code works correctly on FASTA sequences, check the following

In [ ]:
### KKKK+ should not exist in peptide sequences
len(fasta_peptides[fasta_peptides['Peptide'].str.contains("KKKK", na=False)])

0

In [ ]:
### should not allow peptides with "KKK"
len(fasta_peptides[fasta_peptides['Peptide'].str.contains("KKK", na=False)])

0

In [ ]:
### should not allow peptides with "KK"
len(fasta_peptides[fasta_peptides['Peptide'].str.contains("KK", na=False)])

0

In [ ]:
### RRRR+ should not exist in peptide sequences
len(fasta_peptides[fasta_peptides['Peptide'].str.contains("RRRR", na=False)])

0

In [ ]:
### should not allow peptides with "RRR"
len(fasta_peptides[fasta_peptides['Peptide'].str.contains("RRR", na=False)])

0

Desired file display

In [ ]:
### file saved as fasta_peptides

In [9]:
fasta_peptides = pd.read_table('/content/drive/MyDrive/Colab Notebooks/peptide ml detection/data/fasta_peptides.tsv')
fasta_peptides.head()

,Protein,Peptide,Cleavage,Prediction,Probability,Length
0,P18031,EFEQIDK,1,1,0.5,435
1,O15371,EEMDFPQLMK,1,1,0.5,548
2,Q15084,DVIELTDDSFDK,1,1,0.5,440
3,P0DMV8,FGDPVVQSDMK,1,1,0.5,641
4,P48735,DIFQEIFDK,1,1,0.5,452
